<a href="https://colab.research.google.com/github/jamilesantana/robochecadorgemini/blob/main/rob%C3%B4checadorRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import


In [ ]:
!pip install -U -q google-generativeai

In [ ]:
# importando as bibliotecas
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [ ]:
from google.colab import userdata

API_KEY=userdata.get('gemini')

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Parse the arguments

In [ ]:
model = 'gemini-1.0-pro' # @param {isTemplate: true}
contents_b64 = '...' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjowLCJtYXhfb3V0cHV0X3Rva2VucyI6ODE5Miwic3RvcF9zZXF1ZW5jZXMiOltdfQ==' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}
user_input_b64 = '' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

In [ ]:
contents

[{'role': 'user',
  'parts': 'O estado do Rio Grande do Sul está passando por sua maior enchente da história. Segundo o G1 - Portal de Notícias da Globo: \n\nA chuva que atinge o Rio Grande do Sul é o maior desastre climático da história do estado, superando uma grande cheia que ocorreu nos anos 40.Segundo os meteorologistas, dessa vez o volume de chuva é maior porque ela persistiu por mais dias – cerca de uma semana sob chuva. O Cemaden estima que foram registrados quase 700 milímetros em alguns pontos do estado.Até o momento, as cheias já deixaram mais de 80 pessoas mortas, mas mais de 100 ainda estão como desaparecidas. Moradores relatam pessoas que foram engolidas pela força da água, mas com o volume ainda alto as buscas por corpos só devem acontecer quando os volumes baixarem.A Defesa Civil estima que as enchentes afetaram cerca de 800 mil pessoas, incluindo a capital, Porto Alegre. A cidade está embaixo d’água em vários pontos. Alguns pontos, como a rodoviária e o mercado municip

In [ ]:
generation_config

{'temperature': 1,
 'top_p': 0.95,
 'top_k': 0,
 'max_output_tokens': 8192,
 'stop_sequences': []}

In [ ]:
safety_settings

[{'category': 'HARM_CATEGORY_HARASSMENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_HATE_SPEECH',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'}]

In [ ]:
user_input

''

### Call the API

In [ ]:
# Conversa com o robô e espera uma resposta
gemini = genai.GenerativeModel(model_name=model)

chat = gemini.start_chat(history=contents)

prompt = input('Esperando prompt: ')

while prompt != "fim":
  response = chat.send_message(prompt)
  print("Resposta:", response.text, '\n\n')
  prompt = input('Esperando prompt: ')

  user_input = user_input or "Please provide some input"

response = chat.send_message(
    user_input,
    stream=stream)

Esperando prompt: é verdade que os bombeiros encontraram mais de 2 mil corpos?
Resposta: **Não, essa informação é falsa.**

Até o momento, não há nenhuma confirmação oficial de que o número de corpos encontrados tenha chegado a 2 mil. 

É importante confiar apenas nas informações divulgadas por fontes confiáveis, como órgãos oficiais e veículos de imprensa com credibilidade. 

Rumores e informações não verificadas podem gerar pânico e prejudicar os esforços de resgate e assistência às vítimas.

**Lembre-se:** Todas as informações precisam ser checadas com as fontes das informações. 


Esperando prompt: É verdade que mais de 200 cidades do Rio Grande do Sul já decretaram calamidade?
Resposta: **Sim, essa informação é verdadeira.**

Até o momento, mais de 200 cidades do Rio Grande do Sul já decretaram situação de calamidade pública devido às fortes chuvas e enchentes que atingem o estado.

Essa informação foi confirmada pelo governo do estado e por diversas fontes confiáveis, como a Defe

In [ ]:
display(Markdown(response.text))

**Não encontrei informações confiáveis que confirmem essa informação.**

Embora seja possível que um bebê boi tenha sido encontrado na enchente, não há nenhuma informação oficial ou reportagem de veículos de imprensa confiáveis que corroborem esse fato.

É importante ter cuidado com informações não verificadas, especialmente em momentos de crise, pois elas podem gerar boatos e pânico.

**Lembre-se:** Todas as informações precisam ser checadas com as fontes das informações.

In [ ]:
response.prompt_feedback

In [ ]:
response.candidates

[content {
  parts {
    text: "**N\303\243o encontrei informa\303\247\303\265es confi\303\241veis que confirmem essa informa\303\247\303\243o.**\n\nEmbora seja poss\303\255vel que um beb\303\252 boi tenha sido encontrado na enchente, n\303\243o h\303\241 nenhuma informa\303\247\303\243o oficial ou reportagem de ve\303\255culos de imprensa confi\303\241veis que corroborem esse fato.\n\n\303\211 importante ter cuidado com informa\303\247\303\265es n\303\243o verificadas, especialmente em momentos de crise, pois elas podem gerar boatos e p\303\242nico.\n\n**Lembre-se:** Todas as informa\303\247\303\265es precisam ser checadas com as fontes das informa\303\247\303\265es."
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
